#Sales Analysis

In [1]:
!git clone https://github.com/EzraArzi23/Dasar-Dasar-Pemograman.git

Cloning into 'Dasar-Dasar-Pemograman'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 39 (delta 7), reused 29 (delta 5), pack-reused 0
Receiving objects: 100% (39/39), 19.21 KiB | 6.40 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
import pandas as pd
import os
# Mengubah direktori kerja ke direktori repositori yang baru di-clone
os.chdir("Dasar-Dasar-Pemograman")

##Merge csv Files

In [3]:
# Mengakses file atau folder di dalam folder "data"
data_path = "/content/Dasar-Dasar-Pemograman/Salesbulanan"
files = [file for file in os.listdir("/content/Dasar-Dasar-Pemograman/Salesbulanan") if not file.startswith('.')] # Ignore hidden files
all_months_data = pd.DataFrame()

for file in files:
    current_data = pd.read_csv(data_path+"/"+file, delimiter=';', usecols=["ID","Product","Price","QTY","Date","Address"])
    all_months_data = pd.concat([all_months_data, current_data])

all_months_data.to_csv("all_data_copy.csv", index=False)
all_data = pd.read_csv("all_data_copy.csv")
all_data.head()

,ID,Product,Price,QTY,Date,Address
0,111251,B,175,1,13/03/2024,"Jalan Benhil, Jakarta Pusat"
1,111252,A,200,2,14/03/2024,"Jalan Jombang, Tangerang Selatan"
2,111253,A,200,3,15/03/2024,"Jalan Melati, Jakarta Barat"
3,111254,C,165,1,16/03/2024,"Jalan Benhil, Jakarta Pusat"
4,111255,B,175,2,17/03/2024,"Jalan Sabeni, Jakarta Pusat"


##Clean Up Data

In [4]:
# Find NAN
nan_df = all_data[all_data.isna().any(axis=1)]
display(nan_df.head())

all_data = all_data.dropna(how='all')
all_data.head()

,ID,Product,Price,QTY,Date,Address


,ID,Product,Price,QTY,Date,Address
0,111251,B,175,1,13/03/2024,"Jalan Benhil, Jakarta Pusat"
1,111252,A,200,2,14/03/2024,"Jalan Jombang, Tangerang Selatan"
2,111253,A,200,3,15/03/2024,"Jalan Melati, Jakarta Barat"
3,111254,C,165,1,16/03/2024,"Jalan Benhil, Jakarta Pusat"
4,111255,B,175,2,17/03/2024,"Jalan Sabeni, Jakarta Pusat"


##Adding Month and Sales Data

In [5]:
all_data['Month'] = all_data['Date'].str[3:5]
all_data['Month'] = all_data['Month'].astype('int32')
all_data.head()

,ID,Product,Price,QTY,Date,Address,Month
0,111251,B,175,1,13/03/2024,"Jalan Benhil, Jakarta Pusat",3
1,111252,A,200,2,14/03/2024,"Jalan Jombang, Tangerang Selatan",3
2,111253,A,200,3,15/03/2024,"Jalan Melati, Jakarta Barat",3
3,111254,C,165,1,16/03/2024,"Jalan Benhil, Jakarta Pusat",3
4,111255,B,175,2,17/03/2024,"Jalan Sabeni, Jakarta Pusat",3


In [6]:
all_data['Sales'] = all_data["QTY"]*all_data["Price"]
all_data.head()

,ID,Product,Price,QTY,Date,Address,Month,Sales
0,111251,B,175,1,13/03/2024,"Jalan Benhil, Jakarta Pusat",3,175
1,111252,A,200,2,14/03/2024,"Jalan Jombang, Tangerang Selatan",3,400
2,111253,A,200,3,15/03/2024,"Jalan Melati, Jakarta Barat",3,600
3,111254,C,165,1,16/03/2024,"Jalan Benhil, Jakarta Pusat",3,165
4,111255,B,175,2,17/03/2024,"Jalan Sabeni, Jakarta Pusat",3,350


##Task1: The Best Month for Sales

In [7]:
#Group Month And Sales
sales_month= all_data['Sales'] .groupby(all_data['Month']).sum()
sales_month = pd.DataFrame(sales_month)
#Group Month And Quantity
quantity_month = all_data["QTY"].groupby(all_data["Month"]).sum()
quantity_month = pd.DataFrame(quantity_month)
#Merge Them
sales_quantity = pd.concat([sales_month,quantity_month],axis=1)
sales_quantity.sort_values('Sales',ascending=False)

,Sales,QTY
Month,,
5,7635,42
6,5985,33
4,5105,29
1,3885,21
3,3220,17
2,3065,16


##Task2: The Best Product for Sales

In [8]:
#Group Product And Quantity
product_quantity = all_data["QTY"].groupby(all_data['Product']).sum()
product_quantity = pd.DataFrame(product_quantity)
#Group Product And Sales
product_sales = all_data["Sales"].groupby(all_data['Product']).sum()
product_sales = pd.DataFrame(product_sales)
#Merge Them
product_group = pd.concat([product_quantity,product_sales],axis=1)
product_group.sort_values('Sales',ascending=False)

,QTY,Sales
Product,,
A,65,13000
B,55,9625
C,38,6270


##Task3: What City had the Highest Number of Sales

In [9]:
all_data["City"] = all_data["Address"].apply(lambda x: x.split(',')[1])
city_sales = all_data['Sales'].groupby(all_data['City']).sum()
city_sales = pd.DataFrame(city_sales)
city_quantity = all_data["QTY"].groupby(all_data['City']).sum()
city_quantity = pd.DataFrame(city_quantity)

city_max = pd.concat([city_sales,city_quantity],axis=1)
city_max.sort_values('Sales',ascending=False)

,Sales,QTY
City,,
Jakarta Selatan,16180,89
Depok,5270,29
Jakarta Pusat,4745,25
Tangerang Selatan,1410,8
Jakarta Barat,1290,7


##Task4: Average Sales per Month

In [10]:
mean_month = all_data.groupby("Month")["Sales"].mean()
mean_month = pd.DataFrame(mean_month)
mean_month.rename(columns = {'Sales':'Mean'})

,Mean
Month,
1,388.5
2,306.5
3,322.0
4,510.5
5,763.5
6,598.5
